In [7]:
from linkedin_api import Linkedin
from LOGIN import USERNAME,PASSWORD

# create an instance of the API client
api = Linkedin(USERNAME,PASSWORD)

# search for profiles based on a keyword
search_results = api.search_people(keywords=['data engineer','data scientist', 'big data engineer'], regions='morocco', industries=['data science', 'big data'], limit=150)

print(len(search_results),'Profile Founded !')

150 Profile Founded !


#### The document dormat will be folowwed
```json
{
    profile_id,
    headline,
    lastName,
    firstName,
    industryName,
    is student,
    location: {geoCountryName, geoLocationName, location.basicLocation.countryCode},
    birthDate{},
    displayPictureUrl,
    skills: [skills.name],
    education [{
        school: {schoolName, active, trackingId},
        fieldOfStudy,
        degreeName,
        timePeriod: {endDate, startDate}
    }],
    experience[{
        geoLocationName
        companyName
        timePeriod:{startDate{}, endDate{}},
        title,
    }],
}
```

In [9]:
from tqdm import tqdm

target_profiles = list()
for item in tqdm(search_results, desc='Extracting', total=150):
    profile = api.get_profile(item.get('urn_id'))
    profile_dict = {}
    profile_dict['profile_id'] = profile.get('profile_id')
    profile_dict['description'] = profile.get('headline')
    profile_dict['lastName'] = profile.get('lastName')
    profile_dict['firstName'] = profile.get('firstName')
    profile_dict['industry'] = profile.get('industryName')
    profile_dict['student'] = profile.get('student')
    profile_dict['birthDate'] = {'month':profile.get('birthDate')['month'], 'day':profile.get('birthDate')['day']} if profile.get('birthDate') else None
    profile_dict['pictureUrl'] = profile.get('displayPictureUrl')
    profile_dict['location'] = {
        'locationName':profile.get('geoLocationName'),
        'country':profile.get('geoCountryName'),
        'countryCode':profile.get('location').get('basicLocation')['countryCode'].upper()
    }
    profile_dict['skills'] = [skill['name'] for skill in profile.get('skills')]
    profile_dict['education'] = [
        {
            'schoolName': school.get('schoolName'),
            'field': school.get('fieldOfStudy'),
            'degree': school.get('degreeName'),
            'period': school.get('timePeriod')
        }
        for school in profile.get('education')
    ]
    profile_dict['experience'] = [
        {
            'title': experience.get('title'),
            'companyName': experience.get('companyName'),
            'location': experience.get('geoLocationName'),
            'period':experience.get('timePeriod'),
        }
        for experience in profile.get('experience')
    ]

    target_profiles.append(profile_dict)

print(len(target_profiles),"extracted profile ...")

Extracting: 100%|██████████| 150/150 [19:23<00:00,  7.76s/it]

150 extracted profile ...


In [10]:
profile_dict

{'profile_id': 'ACoAAC2MVq0Boem_9d6F_URxNHyXiXCy3k19Fzw',
 'description': 'Data Analyst | Data Scientist | ML Engineer  | Big Data Analyst | FinTech Industry | AI',
 'lastName': 'BAGHEL',
 'firstName': 'VICKYSINGH',
 'industry': 'IT Services and IT Consulting',
 'student': False,
 'birthDate': {'month': 6, 'day': 19},
 'pictureUrl': 'https://media.licdn.com/dms/image/D4D35AQF_1Abn5r9YiA/profile-framedphoto-shrink_',
 'location': {'locationName': 'Milan, Lombardy',
  'country': 'Italy',
  'countryCode': 'IT'},
 'skills': ['Data Analysis',
  'Statistics',
  'Data Visualization',
  'Data Science',
  'Machine Learning',
  'Quality Assurance',
  'SAS Programming',
  'Quality Management',
  'Core Java',
  'SAS',
  'R',
  'Tableau',
  'Python (Programming Language)',
  'Hadoop',
  'Microsoft Power BI',
  'SQL',
  'Microsoft Office',
  'Microsoft Excel',
  'R (Programming Language)',
  'Apache Spark',
  'Big Data Analytics',
  'Data Analytics',
  'Customer Analysis',
  'Predictive Analytics',


In [11]:
import json

with open('linkedin_profiles.json','w') as json_file:
    json.dump(target_profiles, json_file)


In [12]:
import pandas as pd

linkedin_df = pd.read_json('linkedin_profiles.json')
linkedin_df.head()

,profile_id,description,lastName,firstName,industry,student,birthDate,pictureUrl,location,skills,education,experience
0,ACoAADAtTckBM-2AYYBPxwPzklWs9lJFXBS2aLI,Big Data Engineer / Data Scientist,Ammour,Abdelkaioume,"Technology, Information and Internet",False,"{'month': 1, 'day': 8}",https://media.licdn.com/dms/image/C4D03AQH0hZr...,"{'locationName': 'Casablanca-Settat', 'country...","[SQL Server Integration Services (SSIS), SQL S...",[{'schoolName': 'Universite Internationale de ...,[]
1,ACoAACU1jvgBWq2iBpftde9WYjn4wrVyWSDpYpo,Big Data Engineer | Data Scientist | Web Devel...,Solaymane,Chamane,IT Services and IT Consulting,False,"{'month': 5, 'day': 11}",https://media.licdn.com/dms/image/C4E03AQHFiVP...,"{'locationName': 'Marrakesh, Marrakesh-Safi', ...","[Cloud Computing, Leadership d’équipe, Méthode...",[{'schoolName': 'Ecole Nationale des Sciences ...,"[{'title': 'IT Manager', 'companyName': 'Dabad..."
2,ACoAACkzrcEBQTHh2uJ3U8HPshsLlB-1JcYgSRU,Data scientist - Ai movement | Um6p,koulla,Mohamed,IT System Data Services,False,None,https://media.licdn.com/dms/image/D4E03AQEBiKd...,"{'locationName': 'Rabat, Rabat-Salé-Kenitra', ...","[Recherche opérationnelle, VRP, Internet des o...","[{'schoolName': 'ENSET Mohammedia', 'field': '...","[{'title': 'Data Scientist', 'companyName': 'A..."
3,ACoAAC_OWVIBSI63soeqme-y0W4zT4kVD4RGoys,Data Scientist,AIT BELLA,Mouhsine,Software Development,False,"{'month': 7, 'day': 16}",https://media.licdn.com/dms/image/C4D03AQG0i8M...,"{'locationName': 'Taroudant, Souss-Massa', 'co...","[Ingénierie, Recherche, Formation, Stratégie, ...",[{'schoolName': 'Faculté des Sciences et Techn...,"[{'title': 'Data Scientist', 'companyName': 'C..."
4,ACoAAC88LAkBKwe1VOXy0dDeLdiXThbGRSrJR1Y,Data Scientist | Big Data Engineer | BI consul...,Fadoul,Abdellah,Research Services,False,None,https://media.licdn.com/dms/image/D5635AQFDBqR...,{'locationName': 'Casablanca Metropolitan Area...,"[Machine Learning, Deep Learning, Big Data, Py...",[{'schoolName': 'Université Hassan II Mohammed...,[]


In [13]:
linkedin_df.shape

(150, 12)

In [15]:
linkedin_df.to_csv("linkedin_profiles.csv")

In [16]:
pd.read_csv("linkedin_profiles.csv")

,Unnamed: 0,profile_id,description,lastName,firstName,industry,student,birthDate,pictureUrl,location,skills,education,experience
0,0,ACoAADAtTckBM-2AYYBPxwPzklWs9lJFXBS2aLI,Big Data Engineer / Data Scientist,Ammour,Abdelkaioume,"Technology, Information and Internet",False,"{'month': 1, 'day': 8}",https://media.licdn.com/dms/image/C4D03AQH0hZr...,"{'locationName': 'Casablanca-Settat', 'country...","['SQL Server Integration Services (SSIS)', 'SQ...",[{'schoolName': 'Universite Internationale de ...,[]
1,1,ACoAACU1jvgBWq2iBpftde9WYjn4wrVyWSDpYpo,Big Data Engineer | Data Scientist | Web Devel...,Solaymane,Chamane,IT Services and IT Consulting,False,"{'month': 5, 'day': 11}",https://media.licdn.com/dms/image/C4E03AQHFiVP...,"{'locationName': 'Marrakesh, Marrakesh-Safi', ...","['Cloud Computing', 'Leadership d’équipe', 'Mé...",[{'schoolName': 'Ecole Nationale des Sciences ...,"[{'title': 'IT Manager', 'companyName': 'Dabad..."
2,2,ACoAACkzrcEBQTHh2uJ3U8HPshsLlB-1JcYgSRU,Data scientist - Ai movement | Um6p,koulla,Mohamed,IT System Data Services,False,NaN,https://media.licdn.com/dms/image/D4E03AQEBiKd...,"{'locationName': 'Rabat, Rabat-Salé-Kenitra', ...","['Recherche opérationnelle', 'VRP', 'Internet ...","[{'schoolName': 'ENSET Mohammedia', 'field': '...","[{'title': 'Data Scientist', 'companyName': 'A..."
3,3,ACoAAC_OWVIBSI63soeqme-y0W4zT4kVD4RGoys,Data Scientist,AIT BELLA,Mouhsine,Software Development,False,"{'month': 7, 'day': 16}",https://media.licdn.com/dms/image/C4D03AQG0i8M...,"{'locationName': 'Taroudant, Souss-Massa', 'co...","['Ingénierie', 'Recherche', 'Formation', 'Stra...",[{'schoolName': 'Faculté des Sciences et Techn...,"[{'title': 'Data Scientist', 'companyName': 'C..."
4,4,ACoAAC88LAkBKwe1VOXy0dDeLdiXThbGRSrJR1Y,Data Scientist | Big Data Engineer | BI consul...,Fadoul,Abdellah,Research Services,False,NaN,https://media.licdn.com/dms/image/D5635AQFDBqR...,{'locationName': 'Casablanca Metropolitan Area...,"['Machine Learning', 'Deep Learning', 'Big Dat...",[{'schoolName': 'Université Hassan II Mohammed...,[]
...,...,...,...,...,...,...,...,...,...,...,...,...,...
145,145,ACoAACmVakkBzlioDTb9OxZi56jWP7mrye5qoqw,Data Scientist chez Stellantis,Fadili,Anouar,Motor Vehicle Manufacturing,False,NaN,https://media.licdn.com/dms/image/C4E03AQFhc64...,"{'locationName': 'Rabat, Rabat-Salé-Kenitra', ...","['Big Data', 'Analytique', 'Science des donnée...",[{'schoolName': 'Universite Internationale de ...,"[{'title': 'Data scientist', 'companyName': 'S..."
146,146,ACoAACH2sXoBKx_rFSdCcbArLTXJsJARg6Qzn7Q,BI & Big Data Engineer || Data analyst || Data...,Guertite,Soufiane,Business Consulting and Services,False,NaN,https://media.licdn.com/dms/image/D4E03AQFt2n-...,"{'locationName': 'Prefecture of Casablanca, Ca...","['Data mining', 'Machine learning', 'Data Visu...","[{'schoolName': 'FST Settat', 'field': 'Busine...","[{'title': 'Consultant Support FAI', 'companyN..."
147,147,ACoAACTfWycBo3OD0pRtGLj6HPbtOKhjMFXiTyQ,BigData engineer / Data Scientist chez BCP2S (...,MOTONNE,Mehdi,Software Development,False,"{'month': 3, 'day': 14}",https://media.licdn.com/dms/image/C5603AQFTL7p...,"{'locationName': 'Casablanca-Settat', 'country...","['Développement Salesforce.com', 'Data mining'...","[{'schoolName': 'Université Ibn Tofail', 'fiel...",[{'title': 'BigData engineer / Data Scientist'...
148,148,ACoAAA-iW1EBBYCBVENldP4O4Olq46LL2_H6B1Y,Data scientist | ML Engineer | GCP Certified DE,Mestour,Ziyad,IT Services and IT Consulting,False,NaN,https://media.licdn.com/dms/image/C5603AQGhlIi...,"{'locationName': 'Fontenay-sous-Bois, Île-de-F...","['Google Cloud Platform (GCP)', 'Machine Learn...",[{'schoolName': 'Universite Internationale de ...,"[{'title': 'Data Scientist', 'companyName': 'G..."
